## Dyanmic Attributes and Properties

### Data Wrangling

In [1]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = f'Downloading {URL} to {JSON}'
        warnings.warn(msg)
        
        with urlopen(URL) as remote, open(JSON, 'wb', encoding="utf8") as local:
            local.write(remote.read())
    
    with open(JSON, encoding="utf8") as fp:
        return json.load(fp)

In [2]:
feed = load()

In [3]:
feed['Schedule'].keys()

dict_keys(['conferences', 'events', 'speakers', 'venues'])

In [4]:
for k,v in feed['Schedule'].items():
    print(f'{len(v):3} {k}')

  1 conferences
494 events
357 speakers
 53 venues


In [5]:
feed['Schedule']['conferences']

[{'serial': 115}]

In [6]:
feed['Schedule']['events'][0]

{'serial': 33451,
 'name': 'Migrating to the Web Using Dart and Polymer - A Guide for Legacy OOP Developers',
 'event_type': '40-minute conference session',
 'time_start': '2014-07-23 17:00:00',
 'time_stop': '2014-07-23 17:40:00',
 'venue_serial': 1458,
 'description': 'The web development platform is massive. With tons of libraries, frameworks and concepts out there, it might be daunting for the &#39;legacy&#39; developer to jump into it.\r\n\r\nIn this presentation we will introduce Google Dart &amp; Polymer. Two hot technologies that work in harmony to create powerful web applications using concepts familiar to OOP developers.',
 'website_url': 'https://conferences.oreilly.com/oscon/oscon2014/public/schedule/detail/33451',
 'speakers': [149868],
 'categories': ['Emerging Languages']}

Accessing the values using above syntax is difficult. Following class can be used to access the values similar to **JavaScript** *(feed.Schedule.events)*

`__getattr__` method can be used to retrive the attributes using `dot and letters`. This method is invoked only if the usual process fails to retrieve an attribte

In [7]:
from collections import abc
from keyword import iskeyword

class FrozenJSON:
    def __init__(self, mapping):
        self.__data = {}
        for k,v in mapping.items():
            if iskeyword(k):
                k += '_'
            self.__data[k] = v
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])
        
    
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

1. Build a dictionary from *mapping* argument. It also checks if the key is python keyword. If yes then it append an **underscore _** to it.

2. `__getattr__` is called when there is no attribute of *name* present.

    a. If *name* matches an attribute of instance `__data`, return that.
    
    b. Else fetch the item with the key name from `self.__data` and return the result of `build()` method on that.


3. `build()` is an alternative constructor built using `classmethod`.

    a. If obj is mapping, build a `FrozenJSON` with it.
    
    b. If it's mutable sequence, then build list.
    
    c. If not either of these, return the obj as it is.

In [8]:
raw_feed = load()
feed = FrozenJSON(raw_feed)

In [9]:
len(feed.Schedule.speakers)

357

In [10]:
for k,v in feed.Schedule.items():
    print(f'{len(v):3} {k}')

  1 conferences
494 events
357 speakers
 53 venues


In [11]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [12]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [13]:
grad.class_

1982

### Object creation with `__new__`

Special method that actually constructs an instance is `__new__`. It's a classmethod without a decorated and it must return an instance.

This instance will in turn be passed as first argument *self* of **\_\_init__()**.

**\_\_init__()** is actaully forbidden from returning anything, it's just an `initializer`. The real constructor is `__new__`. The `__new__` method can also return an instance of different class

In [17]:
from collections import abc
from keyword import iskeyword

class FrozenJSON:
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls[item] for item in arg]
        else:
            return arg
    
    def __init__(self, mapping):
        self.__data = {}
        for k,v in mapping.items():
            if iskeyword(k):
                k += '_'
            self.__data[k] = v
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])
        

Here `__new__` method gets the class as first argument. 

Expression `super().__new__(cls)` `object.__new__(FrozenJSON)`. Hence the `__class__` attribute of new instance will hold reference to **FrozenJSON**

In [20]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [22]:
grad.class_

1982

### Using a property for attribute validation